In [6]:
import statisUtils as utils
import pandas as pd
import numpy as np
import random
from scipy.stats import loguniform, lognorm
from scipy import stats
import time
import math

import ABCSampler as abcSampler


In [14]:
# df = pd.read_csv('data/MEF_QC_all.csv', header=None) # gene x cells
df = pd.read_csv('data/c57_genes_count.csv', header=None, index_col=[0]) # gene x cells

# data_all = np.array(df, dtype=int)
# data_all

In [15]:
df

,1,2,3,4,5,6,7,8,9,10,...,673,674,675,676,677,678,679,680,681,682
0,,,,,,,,,,,,,,,,,,,,,
Mrpl15,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,NaN,...,2.0,NaN,0.0,NaN,0.0,3.0,13.0,0.0,0.0,12.0
4732440D04Rik,0.0,0.0,NaN,3.0,2.0,0.0,11.0,4.0,0.0,0.0,...,1.0,3.0,2.0,3.0,0.0,4.0,6.0,11.0,2.0,0.0
Cops5,0.0,15.0,NaN,24.0,9.0,10.0,NaN,16.0,23.0,51.0,...,0.0,4.0,0.0,23.0,5.0,0.0,0.0,6.0,NaN,6.0
Arfgef1,4.0,0.0,NaN,0.0,NaN,0.0,10.0,0.0,0.0,0.0,...,1.0,0.0,5.0,NaN,0.0,9.0,0.0,9.0,0.0,0.0
Tram1,20.0,11.0,15.0,12.0,12.0,27.0,35.0,9.0,44.0,8.0,...,5.0,6.0,0.0,4.0,5.0,1.0,8.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Exosc7,2.0,11.0,40.0,2.0,1.0,4.0,8.0,1.0,4.0,9.0,...,3.0,1.0,1.0,0.0,0.0,0.0,2.0,4.0,26.0,1.0
Lars2,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Sacm1l,0.0,0.0,0.0,6.0,4.0,2.0,4.0,0.0,1.0,0.0,...,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,2.0


[ 0.  0.  0.  0.  2.  0.  3.  0.  0. nan  0. 13.  0.  0.  0.  8.  9.  0.
  0.  0.  0.  0.  0.  4. nan  3. nan nan  0.  0.  0.  0.  0. 11.  1.  0.
  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0.  0.
  0.  3.  0.  0.  0.  0.  0.  0.  8.  0.  0.  0.  4.  7.  0. 14.  0.  0.
  2.  0.  0.  0.  0.  0. 14.  0.  0.  1.  0.  0.  0. nan nan  1.  0. nan
 nan  0.  6. nan  6. nan  0.  0.  0. nan nan  3.  4.  4.  0. 15. nan 15.
  0. 16.  2.  0.  0.  0.  5.  1.  0.  0.  0.  0.  2.  0.  0.  0. nan 13.
  0. nan  5.  3. nan 12.  6. nan nan  1. nan  1.  0.  1.  0.  7.  0.  0.
  6.  4.  5.  3.  0.  0.  0.  0.  1.  0.  1. nan nan  0.  0.  0.  1.  1.
 nan nan  0.  0.  0. nan  0. 23.  0.  0.  5.  0.  3.  0.  3.  0.  3.  0.
  0.  0. nan  9. 13.  0.  0.  0.  1.  1.  2. nan  0.  0.  0. 19.  6.  3.
  0. nan  0.  0. 12.  0. 14.  2.  0. nan  0.  0. 17. nan  1.  0.  1.  0.
  0.  3.  0.  0.  0.  0. nan  0.  1.  0.  0.  0.  3. nan  0.  6.  0. nan
  0.  0.  0. nan  5.  2.  5. nan  0.  0.  0.  0.  0

In [8]:
def lognpdf(x, mu, sigma):
    shape  = sigma
    loc    = 0
    scale  = np.exp(mu)
    return lognorm.pdf(x, shape, loc, scale)

In [9]:
count = 1
for for index, row in df.iterrows():
    start_processing = time.time()
    print(f"Start Processing")
    
    gene_name = index
    data = np.array(row)
    
    gene = data[data>=0]
    k = int(0.95*len(gene))
    gene = gene[np.argpartition(gene, k)[:k]]
    gene = np.sort(gene)
    
    data_mean = np.mean(gene);
    data_var = np.var(gene);
    data_noise = data_var/(data_mean**2);
    
    statis_data = utils.statisData(gene);
    
    rho = lambda s: np.sqrt(
                np.sum(
                    np.log(statis_data/(s))**2
                )
    )
    
    
    f = lambda k: utils.statisGTM(k,4)
    N = 200 # the number of particles
    T = 5 # number of rounds
    epsilon = 1# a sequence of discrepancy acceptance thresholds
    
#     prior = lambda: np.array([
#                 random.uniform(0,5), # kon ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # ron ~ logU[-1,1] base 10
#                 random.uniform(0,5), # koff ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # roff ~ logU[-1,1] base 10 
#                 random.uniform(0,50), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
#                 1]) # rdeg=1
    prior = lambda: np.array([
                5 * np.random.uniform(), # kon ~ U[0,5]
                10**random.uniform(-1,1), # ron ~ logU[-1,1] base 10
                5 * np.random.uniform(), # koff ~ U[0,5]
                10**random.uniform(-1,1), # roff ~ logU[-1,1] base 10 
                100 * np.random.uniform(), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
                1]) # rdeg=1
    
    proposal_sigma = 0.2
    
    proposal = lambda x: np.random.lognormal(x,proposal_sigma)
    
    proposal_pdf = lambda kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior: \
    lognpdf(mu_post,np.log(mu_prior),proposal_sigma) * lognpdf(kon_post,np.log(kon_prior),proposal_sigma) \
    * lognpdf(ron_post,np.log(ron_prior),proposal_sigma) * lognpdf(koff_post,np.log(koff_prior),proposal_sigma) \
    * lognpdf(roff_post,np.log(roff_prior),proposal_sigma) 
    
    
#     result, flag = abcSampler.ABCRejectionSampler(N,prior,f,rho,epsilon,T,count)
#     result = np.load('data/gene_1_prior.npy', allow_pickle='True')
    
#     result_true = np.load('data/gene_1_prior.npy', allow_pickle='True')
    
#     print(result_true)

    result, flag = abcSampler.ABCSMCSampler(N,prior,f,rho,epsilon,T,proposal,proposal_pdf,gene_name)
    

    # Save posterior
    np.save(f'data/posterior/gene_{gene_name}.npy', np.array([count, gene, result], dtype=object))
    
    end_processing = time.time()
    total_processing = end_processing - start_processing
    print(f"Finish Processing, time= {total_processing}")
    
    if count > 1: break
    count+=1

Start Processing
Start Processing
Start Processing
Start Processing Prior: Gene = 3
Finished Processing Prior: Time = 13.72244644165039

Start Processing posterior: Gene = 3
Start Processing Posterior: Gene = 3, T = 2
End Processing Posterior: Gene = 3, T = 2, time = 35.22000694274902

Start Processing Posterior: Gene = 3, T = 3
End Processing Posterior: Gene = 3, T = 3, time = 47.95794725418091

Start Processing Posterior: Gene = 3, T = 4
End Processing Posterior: Gene = 3, T = 4, time = 49.334800004959106

Start Processing Posterior: Gene = 3, T = 5
End Processing Posterior: Gene = 3, T = 5, time = 45.4396858215332

Finished Processing posterior: Time = 177.95244002342224

Finish Processing, time= 191.6912281513214


In [5]:
total_processin

NameError: name 'total_processin' is not defined

In [ ]:
data_var

In [ ]:
data_noise

In [ ]:
result[0,0]

In [ ]:
kon_post = 5.30753304760157;
kon_prior = 3.152078539638027;
ron_post = 8.70731191726281;
ron_prior = 9.601789150560395;
koff_post = 3.5623878531212596;
koff_prior = 1.1835791487917824;
roff_post = 4.520511228784241;
roff_prior = 2.138386913508971;
mu_post = 101.10914066538857;
mu_prior = 95.27428156492888;

proposal_pdf(kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior) 

In [ ]:
5 * np.random.uniform()